In [1]:
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from visualisation import *
from utils import *


In [2]:
with open('ferries.json', 'r') as file:
    ferries_data = json.load(file)
print(ferries_data)

{'fragancia': {'pontos_vessel_id': 'mmsi_265558290'}, 'jupiter': {'pontos_vessel_id': 'mmsi_265585310'}, 'merkurius': {'pontos_vessel_id': 'mmsi_265546940'}, 'nina': {'pontos_vessel_id': 'mmsi_265589970'}, 'yxlan': {'pontos_vessel_id': 'imo_8804440'}}


In [15]:
# Path to Excel file provided by Färjerederiet
file_path="./ferry_trips.xlsx"

# Load the Excel file into a Dataframe
ferry_trip_data = pd.read_excel(file_path)

# Get list of all ferries
ferries = list(ferry_trip_data["ferry_name"].unique())

In [ ]:
# Add extra columns to the ferry_trip_data DataFrame
ferry_trip_data["fuelcons_outbound_l"] = None
ferry_trip_data["distance_outbound_nm"] = None
ferry_trip_data["start_time_outbound"] = None
ferry_trip_data["end_time_outbound"] = None

ferry_trip_data["fuelcons_inbound_l"] = None
ferry_trip_data["distance_inbound_nm"] = None
ferry_trip_data["start_time_inbound"] = None
ferry_trip_data["end_time_inbound"] = None

In [5]:
ferries = ["fragancia"]

In [ ]:
# Time bucket
time_bucket = "30 seconds"  # Used to fetch data from PONTOS-HUB
time_bucket_s = 30  # Used to calculate fuel consumption

# Loop through all the ferries
for ferry in tqdm(ferries, desc="Processing ferries"):

    # Get a list of all the days in the time_departure column for the ferry
    days = ferry_trip_data[ferry_trip_data["ferry_name"].str.lower() == ferry][
        "time_departure"
    ].dt.date.unique()

    # Loop through all the days with a progress bar
    for day in tqdm(days, desc="Processing days"):

        # Get departure times for the day
        departure_times = ferry_trip_data[
            (ferry_trip_data["ferry_name"].str.lower() == ferry)
            & (ferry_trip_data["time_departure"].dt.date == day)
        ]["time_departure"]

        # Check if day is after PONTOS-HUB launch date
        if pd.Timestamp(day) <= pd.Timestamp("2023-04-30"):
            continue

        # Fetch vessel data from PONTOS-HUB
        start_of_day = pd.Timestamp(day)
        end_of_day = start_of_day + pd.Timedelta("1 day")
        vessel_data = fetch_vessel_data(
            ferries_data[ferry]["pontos_vessel_id"],
            str(start_of_day),
            str(end_of_day),
            time_bucket=time_bucket,
        )

        # If no vessel data is returned, skip to the next day
        if len(vessel_data) == 0:
            continue

        # Get trips from vessel data
        trips = get_trips_from_vessel_data(vessel_data)

        for departure_time in departure_times:

            # Identify outbound and inbound trips
            outbound_trip = None
            inbound_trip = None
            departure_time = pd.Timestamp(departure_time)
            closest_outbound_trip_index = min(
                range(len(trips)),
                key=lambda i: abs(pd.Timestamp(trips[i]["time"][0]) - departure_time),
            )
            closest_inbound_trip_index = (
                closest_outbound_trip_index + 1
                if closest_outbound_trip_index + 1 < len(trips)
                else closest_outbound_trip_index
            )
            closest_outbound_trip = trips[closest_outbound_trip_index]
            closest_inbound_trip = trips[closest_inbound_trip_index]
            if (
                abs(
                    (
                        pd.Timestamp(closest_outbound_trip["time"][0]) - departure_time
                    ).total_seconds()
                )
                / 60
                <= 5
            ):  # 5 minutes diff is acceptable
                outbound_trip = closest_outbound_trip
                if (
                    pd.Timestamp(closest_outbound_trip["time"][0])
                    - pd.Timestamp(closest_outbound_trip["time"][-1])
                ).total_seconds() / 60 <= 10:  # 10 minutes between first and last point is acceptable
                    inbound_trip = closest_inbound_trip

            distance_outbound_nm = None
            distance_inbound_nm = None
            fuelcons_outbound_l = None
            fuelcons_inbound_l = None
            start_time_outbound = None
            end_time_outbound = None
            start_time_inbound = None
            end_time_inbound = None

            if outbound_trip is not None:

                # Determine the start and end time for the outbound trip
                start_time_outbound = pd.Timestamp(outbound_trip["time"][0])
                end_time_outbound = pd.Timestamp(outbound_trip["time"][-1])

                # Calculate the total distance for the outbound trip
                distance_outbound_nm = sum(
                    [
                        haversine(
                            outbound_trip["path"][i], outbound_trip["path"][i - 1]
                        )
                        / 1_852
                        for i in range(1, len(outbound_trip["path"]))
                    ]
                )

                # Calculate the total fuel consumption for the outbound trip
                for key in outbound_trip.keys():
                    if "fuelcons_lph" in key:
                        if any([value == None for value in outbound_trip[key]]):
                            break
                        if fuelcons_outbound_l is None:
                            fuelcons_outbound_l = 0
                        fuelcons_outbound_l += sum(
                            [
                                fuelcons_lph * time_bucket_s / 3_600
                                for fuelcons_lph in outbound_trip[key]
                            ]
                        )

            if inbound_trip is not None:

                # Determine the start and end time for the inbound trip
                start_time_inbound = pd.Timestamp(inbound_trip["time"][0])
                end_time_inbound = pd.Timestamp(inbound_trip["time"][-1])

                # Calculate the total distance for the inbound trip
                distance_inbound_nm = sum(
                    [
                        haversine(inbound_trip["path"][i], inbound_trip["path"][i - 1])
                        / 1_852
                        for i in range(1, len(inbound_trip["path"]))
                    ]
                )

                # Calculate the total fuel consumption for the inbound trip
                for key in inbound_trip.keys():
                    if "fuelcons_lph" in key:
                        if any([value == None for value in inbound_trip[key]]):
                            break
                        if fuelcons_inbound_l is None:
                            fuelcons_inbound_l = 0
                        fuelcons_inbound_l += sum(
                            [
                                fuelcons_lph * time_bucket_s / 3_600
                                for fuelcons_lph in inbound_trip[key]
                            ]
                        )

            # Update ferry_trip_data DataFrame for the matching row
            row_mask = (ferry_trip_data["ferry_name"].str.lower() == ferry) & (
                ferry_trip_data["time_departure"] == departure_time
            )
            ferry_trip_data.loc[row_mask, "fuelcons_outbound_l"] = fuelcons_outbound_l
            ferry_trip_data.loc[row_mask, "distance_outbound_nm"] = distance_outbound_nm
            ferry_trip_data.loc[row_mask, "start_time_outbound"] = start_time_outbound
            ferry_trip_data.loc[row_mask, "end_time_outbound"] = end_time_outbound

            ferry_trip_data.loc[row_mask, "fuelcons_inbound_l"] = fuelcons_inbound_l
            ferry_trip_data.loc[row_mask, "distance_inbound_nm"] = distance_inbound_nm
            ferry_trip_data.loc[row_mask, "start_time_inbound"] = start_time_inbound
            ferry_trip_data.loc[row_mask, "end_time_inbound"] = end_time_inbound

Ferry: fragancia


Processing days:   0%|          | 0/1 [00:00<?, ?it/s]

/home/felipe/miniconda3/envs/hack-a-fleet-v2/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [14]:
trv_data.iloc[7208].to_csv('trv_data_subset.csv', index=False)

In [ ]:
df["avg_time"] = pd.to_datetime(df["avg_time"], format="ISO8601")
df["time"] = pd.to_datetime(df["time"], format="ISO8601")

In [12]:
trv_data.loc[row_mask]

,route_id,route_name,ferry_name,ferry_id,terminal_departure,terminal_arrival,time_departure,cars_outbound,trucks_outbound,trucks_with_trailer_outbound,...,passenger_car_equivalent_outbound,passenger_car_equivalent_inbound,fuelcons_outbound_l,distance_outbound_nm,start_time_outbound,end_time_outbound,fuelcons_inbound_l,distance_inbound_nm,start_time_inbound,end_time_inbound
7208,17,Oxdjupsleden,Fragancia,289,Rindö ...,Värmdö ...,2023-07-29 22:50:00,2,0,0,...,2.0,0.0,3.552028,0.219289,2023-07-29 22:50:44,2023-07-29 22:53:14,3.093528,0.220158,2023-07-29 22:55:14,2023-07-29 22:57:44


In [16]:
df.head()

,bucket,vessel_id,parameter_id,avg_time,avg_value
0,2023-07-29T00:00:00+00:00,mmsi_265558290,enginemain_fuelcons_lph_1,2023-07-29 00:00:14.500000+00:00,0.0
1,2023-07-29T00:00:00+00:00,mmsi_265558290,enginemain_fuelcons_lph_2,2023-07-29 00:00:14.500000+00:00,0.0
2,2023-07-29T00:00:00+00:00,mmsi_265558290,enginemain_fuelcons_lph_3,2023-07-29 00:00:14.500000+00:00,0.0
3,2023-07-29T00:00:00+00:00,mmsi_265558290,enginemain_fuelcons_lph_4,2023-07-29 00:00:14.500000+00:00,0.0
4,2023-07-29T00:00:00+00:00,mmsi_265558290,enginemain_speed_rpm_1,2023-07-29 00:00:14.500000+00:00,0.0


In [59]:
haversine(path[0], path[1])

75.39572078901556

In [17]:
 # Visualize trip clusters
r = plot_paths([trip["path"] for trip in trips], get_cluster_colors(labels))
r.to_html()

In [5]:
# Get a list of the ferries in the data
excel_ferries = list(data["farjenamn"].unique())
print(excel_ferries)

['Fragancia', 'Jupiter', 'Merkurius', 'Nina', 'Yxlan']


In [34]:
ferries['fragancia']['pontos_id']

'mmsi_265558290'

In [35]:
# Query a trip in the Excel file
averaged_vessel_data = fetch_vessel_data(ferries['fragancia']['pontos_id'],"2023-05-01T07:20:00","2023-05-01T08:20:00",['lat','lon'], time_bucket='5 minutes')


In [36]:
vessel_data = transform_averaged_vessel_data_to_dataframe(averaged_vessel_data)
vessel_data.head()

parameter_id,positioningsystem_latitude_deg_1,positioningsystem_longitude_deg_1
avg_time,,
2023-05-01 07:22:29.500000+00:00,59.395435,18.444900
2023-05-01 07:27:29.500000+00:00,59.396061,18.447614
2023-05-01 07:32:29.500000+00:00,59.395233,18.441523
2023-05-01 07:37:29.500000+00:00,59.395284,18.441312
2023-05-01 07:42:29.500000+00:00,59.395282,18.441305


In [54]:
vessel_track = vessel_data[["positioningsystem_latitude_deg_1", "positioningsystem_longitude_deg_1"]].values.tolist()
center_point = list(vessel_data[["positioningsystem_longitude_deg_1", "positioningsystem_latitude_deg_1"]].mean())


In [57]:
view_state = pdk.ViewState(latitude=center_point[1], longitude=center_point[0], zoom=10)

layer = pdk.Layer(
    type="ScatterplotLayer",
    data=vessel_data,
    get_position=["positioningsystem_longitude_deg_1","positioningsystem_latitude_deg_1"],
    pickable=True,
    get_fill_color=(255, 232, 0),
    get_radius=10,
)

r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "{name}"})
r.to_html("path_layer.html")

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3154866346.py, line 6)

In [27]:
import pandas as pd
import pydeck as pdk

DATA_URL = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/website/bart-lines.json"
df = pd.read_json(DATA_URL)


def hex_to_rgb(h):
    h = h.lstrip("#")
    return tuple(int(h[i : i + 2], 16) for i in (0, 2, 4))


df["color"] = df["color"].apply(hex_to_rgb)


view_state = pdk.ViewState(latitude=37.782556, longitude=-122.3484867, zoom=10)

layer = pdk.Layer(
    type="PathLayer",
    data=df,
    pickable=True,
    get_color="color",
    width_scale=20,
    width_min_pixels=2,
    get_path="path",
    get_width=5,
)

r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "{name}"})
r.to_html("path_layer.html")

In [28]:
df

,name,color,path
0,Richmond - Millbrae,"(237, 28, 36)","[[-122.3535851, 37.9360513], [-122.3179784, 37..."
1,Fremont - Richmond,"(250, 166, 26)","[[-122.353165, 37.936887], [-122.317269, 37.92..."
2,Pittsburg/Bay Point - SFO,"(255, 232, 0)","[[-121.945154, 38.018914], [-122.024597, 38.00..."
3,Dublin/Pleasanton - Daly City,"(0, 174, 239)","[[-121.900367, 37.701695], [-121.928099, 37.69..."
4,Fremont - Daly City,"(77, 184, 72)","[[-121.9764, 37.557355], [-122.017867, 37.5912..."


In [36]:
response = query_pontos('https://pontos.ri.se/api/vessel_data_30_seconds_average?parameter_id=eq.positioningsystem_sog_kn_1&bucket=gte.2023-08-01T00:00:00&bucket=lt.2023-08-01T00:02:00&vessel_id=eq.mmsi_265585310&limit=10000')
response

[{'bucket': '2023-08-01T00:00:00+00:00',
  'vessel_id': 'mmsi_265585310',
  'parameter_id': 'positioningsystem_sog_kn_1',
  'avg_time': '2023-08-01T00:00:14.5+00:00',
  'avg_value': 0.36},
 {'bucket': '2023-08-01T00:00:30+00:00',
  'vessel_id': 'mmsi_265585310',
  'parameter_id': 'positioningsystem_sog_kn_1',
  'avg_time': '2023-08-01T00:00:44.5+00:00',
  'avg_value': 3.0033333433333333},
 {'bucket': '2023-08-01T00:01:00+00:00',
  'vessel_id': 'mmsi_265585310',
  'parameter_id': 'positioningsystem_sog_kn_1',
  'avg_time': '2023-08-01T00:01:14.5+00:00',
  'avg_value': 5.56666669},
 {'bucket': '2023-08-01T00:01:30+00:00',
  'vessel_id': 'mmsi_265585310',
  'parameter_id': 'positioningsystem_sog_kn_1',
  'avg_time': '2023-08-01T00:01:44.5+00:00',
  'avg_value': 6.8533333333333335}]

In [34]:
response = query_pontos('https://pontos.ri.se/api/vessel_data?parameter_id=eq.positioningsystem_sog_kn_1&time=gte.2023-08-01T00:00:00&time=lt.2023-08-01T00:10:00&vessel_id=eq.mmsi_265585310&limit=10000')
total = 0
for row in response:
    total += float(row['value'])
print(f"Average value is {total / len(response)}")
response[0]

Average value is 5.1121664906666755


{'time': '2023-08-01T00:09:59+00:00',
 'vessel_id': 'mmsi_265585310',
 'parameter_id': 'positioningsystem_sog_kn_1',
 'value': '8.2'}

In [70]:
positions = get_vessel_data(ferries["jupiter"]["pontos_id"], limit=1000)

https://pontos.ri.se/api/vessel_data?parameter_id=ilike.*position*&time=gte.2023-10-29&time=lt.2023-10-30&vessel_id=eq.mmsi_265585310&limit=1000


In [62]:

def aggregate_vessel_data(data):
    # Convert the list of dictionaries into a DataFrame
    df = pd.DataFrame(data)

    # Convert the 'time' column to datetime format
    df['time'] = pd.to_datetime(df['time'])

    # Pivot the DataFrame to have parameter_ids as columns, time as rows
    pivot_df = df.pivot_table(index='time', columns='parameter_id', values='value', aggfunc='first')

    # Resample to every 5 seconds, using the last available value
    resampled_df = pivot_df.resample('10s').last().ffill()

    return resampled_df

In [63]:
aggregate_vessel_data(positions)

parameter_id,positioningsystem_cog_deg_1,positioningsystem_latitude_deg_1,positioningsystem_longitude_deg_1,positioningsystem_sog_kn_1
time,,,,
2023-10-29 23:55:50+00:00,260.1,59.32089,18.099554,0
2023-10-29 23:56:00+00:00,289.8,59.32088,18.099556,0
2023-10-29 23:56:10+00:00,209.6,59.320877,18.099556,0
2023-10-29 23:56:20+00:00,214.4,59.320877,18.099556,0
2023-10-29 23:56:30+00:00,196.2,59.320873,18.099558,0
2023-10-29 23:56:40+00:00,196.1,59.320873,18.099567,0
2023-10-29 23:56:50+00:00,227.7,59.320873,18.099564,0
2023-10-29 23:57:00+00:00,190,59.320873,18.099554,0
2023-10-29 23:57:10+00:00,229.9,59.320873,18.099539,0


In [48]:


# Calculate the date range for each vessel
date_ranges = data.groupby('farjenamn')['datumtid'].agg(['min', 'max'])
print(date_ranges)

                          min                 max
farjenamn                                        
Fragancia 2023-03-01 05:44:00 2024-02-29 22:20:00
Jupiter   2023-03-01 00:00:00 2024-02-29 23:00:00
Merkurius 2023-03-01 00:00:00 2024-02-29 23:30:00
Nina      2023-03-01 00:00:00 2024-02-29 21:30:00
Yxlan     2023-03-01 05:30:00 2024-02-29 22:30:00


In [71]:
positions

[{'time': '2023-10-29T23:59:59+00:00',
  'vessel_id': 'mmsi_265585310',
  'parameter_id': 'positioningsystem_latitude_deg_1',
  'value': '59.320873'},
 {'time': '2023-10-29T23:59:59+00:00',
  'vessel_id': 'mmsi_265585310',
  'parameter_id': 'positioningsystem_longitude_deg_1',
  'value': '18.09956'},
 {'time': '2023-10-29T23:59:59+00:00',
  'vessel_id': 'mmsi_265585310',
  'parameter_id': 'positioningsystem_sog_kn_1',
  'value': '0'},
 {'time': '2023-10-29T23:59:59+00:00',
  'vessel_id': 'mmsi_265585310',
  'parameter_id': 'positioningsystem_cog_deg_1',
  'value': '205.5'},
 {'time': '2023-10-29T23:59:58+00:00',
  'vessel_id': 'mmsi_265585310',
  'parameter_id': 'positioningsystem_latitude_deg_1',
  'value': '59.320873'},
 {'time': '2023-10-29T23:59:58+00:00',
  'vessel_id': 'mmsi_265585310',
  'parameter_id': 'positioningsystem_longitude_deg_1',
  'value': '18.09956'},
 {'time': '2023-10-29T23:59:58+00:00',
  'vessel_id': 'mmsi_265585310',
  'parameter_id': 'positioningsystem_sog_kn_1

In [70]:
datavessel_data = fetch_vessel_data("imo_7943354","2024-11-11T00:00:00","2024-11-13T00:00:00", time_bucket="5 minutes")

In [71]:
datavessel_data

[{'bucket': '2024-11-11T00:00:00+00:00',
  'vessel_id': 'imo_7943354',
  'parameter_id': 'boiler_fuelcons_lph_1',
  'avg_time': '2024-11-11T00:02:29.5+00:00',
  'avg_value': 0.0},
 {'bucket': '2024-11-11T00:00:00+00:00',
  'vessel_id': 'imo_7943354',
  'parameter_id': 'electricalbus_shorepower_kw_1',
  'avg_time': '2024-11-11T00:02:29.5+00:00',
  'avg_value': 7.772294142},
 {'bucket': '2024-11-11T00:00:00+00:00',
  'vessel_id': 'imo_7943354',
  'parameter_id': 'engineaux_power_kw_1',
  'avg_time': '2024-11-11T00:02:29.5+00:00',
  'avg_value': 0.0},
 {'bucket': '2024-11-11T00:00:00+00:00',
  'vessel_id': 'imo_7943354',
  'parameter_id': 'enginemain_fuelcons_lph_1',
  'avg_time': '2024-11-11T00:02:29.5+00:00',
  'avg_value': 0.0},
 {'bucket': '2024-11-11T00:00:00+00:00',
  'vessel_id': 'imo_7943354',
  'parameter_id': 'enginemain_speed_rpm_1',
  'avg_time': '2024-11-11T00:02:29.5+00:00',
  'avg_value': 0.0},
 {'bucket': '2024-11-11T00:00:00+00:00',
  'vessel_id': 'imo_7943354',
  'parame